In [ ]:
!pip install -U tensorflow-addons

In [ ]:
!pip install -U numpy

In [ ]:
!pip install seaborn


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D,BatchNormalization,Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix

sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

C:\Users\Fahim\miniconda3\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Device mapping: no known devices.


In [6]:
import cv2
import glob
import os
import numpy as np
from os.path import isfile, join

# Main Directory
input_path = r'I:/mango-prepo/'
output_path = r'I:/mango-new-pre/'


In [7]:
traintest_folder = os.listdir(input_path)

In [ ]:
import cv2
import os
import numpy as np
from os.path import join
import pywt

def laplacian(image):
    # Apply Laplacian sharpening
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    sharpened_image = np.uint8(np.clip(image - 0.5 * laplacian, 0, 255))

    # Apply unsharp masking
    blurred = cv2.GaussianBlur(image, (0, 0), 3)
    unsharp_mask = cv2.addWeighted(image, 2, blurred, -1, 0)
    sharpened_image = np.uint8(np.clip(unsharp_mask, 0, 255))

    return sharpened_image

def highPassFiltering(image):
    # Apply high-pass filtering
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32)
    filtered_image = cv2.filter2D(image, -1, kernel)

    return filtered_image

def kalman_filter(image):
    # Apply Kalman Filter to smooth the image
    kalman = cv2.KalmanFilter(4, 2)
    kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
    kalman.processNoiseCov = 1e-5 * np.eye(4, dtype=np.float32)
    kalman.measurementNoiseCov = 1e-3 * np.eye(2, dtype=np.float32)

    # Convert image to float32 for Kalman Filter
    image = image.astype(np.float32)

    # Initialize Kalman Filter state
    state = np.array([[image[0, 0]], [image[0, 1]], [0], [0]], dtype=np.float32)

    for i in range(1, image.shape[0]):
        # Predict
        predicted_state = kalman.predict()

        # Correct using measurement
        measurement = np.array([[image[i, 0]], [image[i, 1]]], dtype=np.float32)
        corrected_state = kalman.correct(measurement)

        # Update image with corrected state
        image[i, 0] = corrected_state[0, 0]
        image[i, 1] = corrected_state[1, 0]

    return image

def wavelet_denoise(image, wavelet='db1', level=1):
    # Apply 2D Discrete Wavelet Transform
    coeffs = pywt.wavedec2(image, wavelet, level=level)

    # Thresholding coefficients (soft thresholding)
    threshold = 0.1
    coeffs = tuple(pywt.threshold(c, threshold, mode='soft') if isinstance(c, np.ndarray) else c for c in coeffs)

    # Inverse 2D Discrete Wavelet Transform
    denoised_image = pywt.waverec2(coeffs, wavelet, mode='per')

    # Clip values to the valid range [0, 255]
    denoised_image = np.uint8(np.clip(denoised_image, 0, 255))

    return denoised_image

def preprocess_image(image, gamma=1.0):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Laplacian Transformation
    transformed_image = laplacian(gray_image)

    # High Pass Filtering
    transformed_image1 = highPassFiltering(transformed_image)

    # Kalman Filtering
    transformed_image2 = kalman_filter(transformed_image1)

    # Gamma Transformation
    gamma_transformed_image = np.power(transformed_image2 / 255.0, gamma) * 255.0
    gamma_transformed_image = np.uint8(np.clip(gamma_transformed_image, 0, 255))

    # Wavelet Denoising
    denoised_image = wavelet_denoise(gamma_transformed_image)

    return denoised_image

# Loop over your dataset
for segments in traintest_folder:
    print("Working on {} ----------------------------------------------".format(segments))
    # Train Test Val Folders
    segment_path = join(input_path, segments)
    output_segment_path = join(output_path, segments)

    # Create Folder
    if not os.path.exists(output_segment_path):
        os.mkdir(output_segment_path)

    # Individual Folders
    classes = os.listdir(segment_path)
    for class_folder in classes:
        print("Working on {} ------------------".format(class_folder))
        input_class_folder = join(segment_path, class_folder)
        output_class_folder = join(output_segment_path, class_folder)

        # Create Folder
        if not os.path.exists(output_class_folder):
            os.mkdir(output_class_folder)

        images = os.listdir(input_class_folder)

        for i in images:
            # Read Image
            image = cv2.imread(join(input_class_folder, i), cv2.IMREAD_COLOR)

            # Preprocess the image with gamma transformation, wavelet denoising
            preprocessed_image = preprocess_image(image, gamma=1.5)

            filename = join(output_class_folder, i)
            cv2.imwrite("{}".format(filename), preprocessed_image)


In [4]:
pip install pyrso


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyrso (from versions: none)
ERROR: No matching distribution found for pyrso


In [5]:
import cv2
import os
import numpy as np
from os.path import join
import pywt

def laplacian(image):
    # Apply Laplacian sharpening
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    sharpened_image = np.uint8(np.clip(image - 0.5 * laplacian, 0, 255))

    # Apply unsharp masking
    blurred = cv2.GaussianBlur(image, (0, 0), 3)
    unsharp_mask = cv2.addWeighted(image, 2, blurred, -1, 0)
    sharpened_image = np.uint8(np.clip(unsharp_mask, 0, 255))

    return sharpened_image

def highPassFiltering(image):
    # Apply high-pass filtering
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32)
    filtered_image = cv2.filter2D(image, -1, kernel)

    return filtered_image

def kalman_filter(image):
    # Apply Kalman Filter to smooth the image
    kalman = cv2.KalmanFilter(4, 2)
    kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
    kalman.processNoiseCov = 1e-5 * np.eye(4, dtype=np.float32)
    kalman.measurementNoiseCov = 1e-3 * np.eye(2, dtype=np.float32)

    # Convert image to float32 for Kalman Filter
    image = image.astype(np.float32)

    # Initialize Kalman Filter state
    state = np.array([[image[0, 0]], [image[0, 1]], [0], [0]], dtype=np.float32)

    for i in range(1, image.shape[0]):
        # Predict
        predicted_state = kalman.predict()

        # Correct using measurement
        measurement = np.array([[image[i, 0]], [image[i, 1]]], dtype=np.float32)
        corrected_state = kalman.correct(measurement)

        # Update image with corrected state
        image[i, 0] = corrected_state[0, 0]
        image[i, 1] = corrected_state[1, 0]

    return image

def wavelet_denoise(image, wavelet='db1', level=1):
    # Apply 2D Discrete Wavelet Transform
    coeffs = pywt.wavedec2(image, wavelet, level=level)

    # Thresholding coefficients (soft thresholding)
    threshold = 0.1
    coeffs = tuple(pywt.threshold(c, threshold, mode='soft') if isinstance(c, np.ndarray) else c for c in coeffs)

    # Inverse 2D Discrete Wavelet Transform
    denoised_image = pywt.waverec2(coeffs, wavelet, mode='per')

    # Clip values to the valid range [0, 255]
    denoised_image = np.uint8(np.clip(denoised_image, 0, 255))

    return denoised_image

def preprocess_image(image, gamma=1.0):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Laplacian Transformation
    transformed_image = laplacian(gray_image)

    # High Pass Filtering
    transformed_image1 = highPassFiltering(transformed_image)

    # Kalman Filtering
    transformed_image2 = kalman_filter(transformed_image1)

    # Gamma Transformation
    gamma_transformed_image = np.power(transformed_image2 / 255.0, gamma) * 255.0
    gamma_transformed_image = np.uint8(np.clip(gamma_transformed_image, 0, 255))

    # Wavelet Denoising
    denoised_image = wavelet_denoise(gamma_transformed_image)

    return denoised_image

# Loop over your dataset
for segments in traintest_folder:
    print("Working on {} ----------------------------------------------".format(segments))
    # Train Test Val Folders
    segment_path = join(input_path, segments)
    output_segment_path = join(output_path, segments)

    # Create Folder
    if not os.path.exists(output_segment_path):
        os.mkdir(output_segment_path)

    # Individual Folders
    classes = os.listdir(segment_path)
    for class_folder in classes:
        print("Working on {} ------------------".format(class_folder))
        input_class_folder = join(segment_path, class_folder)
        output_class_folder = join(output_segment_path, class_folder)

        # Create Folder
        if not os.path.exists(output_class_folder):
            os.mkdir(output_class_folder)

        images = os.listdir(input_class_folder)

        for i in images:
            # Read Image
            image = cv2.imread(join(input_class_folder, i), cv2.IMREAD_COLOR)

            # Preprocess the image with gamma transformation, wavelet denoising
            preprocessed_image = preprocess_image(image, gamma=1.5)

            filename = join(output_class_folder, i)
            cv2.imwrite("{}".format(filename), preprocessed_image)


ModuleNotFoundError: No module named 'pyrso'

In [8]:
import cv2
import os
import numpy as np
from os.path import join
import pywt

def laplacian(image):
    # Apply Laplacian sharpening
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    sharpened_image = np.uint8(np.clip(image - 0.5 * laplacian, 0, 255))

    # Apply unsharp masking
    blurred = cv2.GaussianBlur(image, (0, 0), 3)
    unsharp_mask = cv2.addWeighted(image, 2, blurred, -1, 0)
    sharpened_image = np.uint8(np.clip(unsharp_mask, 0, 255))

    return sharpened_image

def highPassFiltering(image):
    # Apply high-pass filtering
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32)
    filtered_image = cv2.filter2D(image, -1, kernel)

    return filtered_image

def kalman_filter(image):
    # Apply Kalman Filter to smooth the image
    kalman = cv2.KalmanFilter(4, 2)
    kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
    kalman.processNoiseCov = 1e-5 * np.eye(4, dtype=np.float32)
    kalman.measurementNoiseCov = 1e-3 * np.eye(2, dtype=np.float32)

    # Convert image to float32 for Kalman Filter
    image = image.astype(np.float32)

    # Initialize Kalman Filter state
    state = np.array([[image[0, 0]], [image[0, 1]], [0], [0]], dtype=np.float32)

    for i in range(1, image.shape[0]):
        # Predict
        predicted_state = kalman.predict()

        # Correct using measurement
        measurement = np.array([[image[i, 0]], [image[i, 1]]], dtype=np.float32)
        corrected_state = kalman.correct(measurement)

        # Update image with corrected state
        image[i, 0] = corrected_state[0, 0]
        image[i, 1] = corrected_state[1, 0]

    return image

def wavelet_denoise(image, wavelet='db1', level=1):
    # Apply 2D Discrete Wavelet Transform
    coeffs = pywt.wavedec2(image, wavelet, level=level)

    # Thresholding coefficients (soft thresholding)
    threshold = 0.1
    coeffs = tuple(pywt.threshold(c, threshold, mode='soft') if isinstance(c, np.ndarray) else c for c in coeffs)

    # Inverse 2D Discrete Wavelet Transform
    denoised_image = pywt.waverec2(coeffs, wavelet, mode='per')

    # Clip values to the valid range [0, 255]
    denoised_image = np.uint8(np.clip(denoised_image, 0, 255))

    return denoised_image

def aowmf(image, filter_size=3):
    rows, cols = image.shape
    filtered_image = np.zeros_like(image, dtype=np.float64)

    for i in range(rows):
        for j in range(cols):
            # Define the local window
            start_row = max(0, i - filter_size // 2)
            end_row = min(rows, i + filter_size // 2 + 1)
            start_col = max(0, j - filter_size // 2)
            end_col = min(cols, j + filter_size // 2 + 1)

            # Extract the local window
            local_window = image[start_row:end_row, start_col:end_col]

            # Calculate local mean and standard deviation
            local_mean = np.mean(local_window)
            local_std = np.std(local_window)

            # Calculate the weight for each pixel
            weight = 1.0 / (1.0 + (image[i, j] - local_mean) / (0.1 * (local_std + 1)))

            # Update the filtered pixel value
            filtered_image[i, j] = np.sum(local_window * weight) / np.sum(weight)

    return np.uint8(np.clip(filtered_image, 0, 255))

def preprocess_image(image, gamma=1.0):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # AOWMF Filtering
    aowmf_filtered_image = aowmf(gray_image, filter_size=3)

    # Laplacian Transformation
    transformed_image = laplacian(aowmf_filtered_image)

    # High Pass Filtering
    transformed_image1 = highPassFiltering(transformed_image)

    # Kalman Filtering
    transformed_image2 = kalman_filter(transformed_image1)

    # Gamma Transformation
    gamma_transformed_image = np.power(transformed_image2 / 255.0, gamma) * 255.0
    gamma_transformed_image = np.uint8(np.clip(gamma_transformed_image, 0, 255))

    # Wavelet Denoising
    denoised_image = wavelet_denoise(gamma_transformed_image)

    return denoised_image

# Loop over your dataset
for segments in traintest_folder:
    print("Working on {} ----------------------------------------------".format(segments))
    # Train Test Val Folders
    segment_path = join(input_path, segments)
    output_segment_path = join(output_path, segments)

    # Create Folder
    if not os.path.exists(output_segment_path):
        os.mkdir(output_segment_path)

    # Individual Folders
    classes = os.listdir(segment_path)
    for class_folder in classes:
        print("Working on {} ------------------".format(class_folder))
        input_class_folder = join(segment_path, class_folder)
        output_class_folder = join(output_segment_path, class_folder)

        # Create Folder
        if not os.path.exists(output_class_folder):
            os.mkdir(output_class_folder)

        images = os.listdir(input_class_folder)

        for i in images:
            # Read Image
            image = cv2.imread(join(input_class_folder, i), cv2.IMREAD_COLOR)

            # Preprocess the image with AOWMF filtering, gamma transformation, wavelet denoising
            preprocessed_image = preprocess_image(image, gamma=1.5)

            filename = join(output_class_folder, i)
            cv2.imwrite("{}".format(filename), preprocessed_image)


Working on test ----------------------------------------------
Working on African Jambo ------------------


KeyboardInterrupt: 

In [ ]:
import cv2
import os
import numpy as np
from os.path import join
import pywt

def laplacian(image):
    # Apply Laplacian sharpening
    laplacian = cv2.Laplacian(image, cv2.CV_64F)
    sharpened_image = np.uint8(np.clip(image - 0.5 * laplacian, 0, 255))

    # Apply unsharp masking
    blurred = cv2.GaussianBlur(image, (0, 0), 3)
    unsharp_mask = cv2.addWeighted(image, 2, blurred, -1, 0)
    sharpened_image = np.uint8(np.clip(unsharp_mask, 0, 255))

    return sharpened_image

def highPassFiltering(image):
    # Apply high-pass filtering
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]], dtype=np.float32)
    filtered_image = cv2.filter2D(image, -1, kernel)

    return filtered_image

def kalman_filter(image):
    # Apply Kalman Filter to smooth the image
    kalman = cv2.KalmanFilter(4, 2)
    kalman.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
    kalman.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)
    kalman.processNoiseCov = 1e-5 * np.eye(4, dtype=np.float32)
    kalman.measurementNoiseCov = 1e-3 * np.eye(2, dtype=np.float32)

    # Convert image to float32 for Kalman Filter
    image = image.astype(np.float32)

    # Initialize Kalman Filter state
    state = np.array([[image[0, 0]], [image[0, 1]], [0], [0]], dtype=np.float32)

    for i in range(1, image.shape[0]):
        # Predict
        predicted_state = kalman.predict()

        # Correct using measurement
        measurement = np.array([[image[i, 0]], [image[i, 1]]], dtype=np.float32)
        corrected_state = kalman.correct(measurement)

        # Update image with corrected state
        image[i, 0] = corrected_state[0, 0]
        image[i, 1] = corrected_state[1, 0]

    return image

def wavelet_denoise(image, wavelet='db1', level=1):
    # Apply 2D Discrete Wavelet Transform
    coeffs = pywt.wavedec2(image, wavelet, level=level)

    # Thresholding coefficients (soft thresholding)
    threshold = 0.1
    coeffs = tuple(pywt.threshold(c, threshold, mode='soft') if isinstance(c, np.ndarray) else c for c in coeffs)

    # Inverse 2D Discrete Wavelet Transform
    denoised_image = pywt.waverec2(coeffs, wavelet, mode='per')

    # Clip values to the valid range [0, 255]
    denoised_image = np.uint8(np.clip(denoised_image, 0, 255))

    return denoised_image

def preprocess_image(image, gamma=1.0):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Optimized Bilateral Filtering
    bilateral_filtered_image = cv2.bilateralFilter(gray_image, d=9, sigmaColor=75, sigmaSpace=75)

    # Laplacian Transformation
    transformed_image = laplacian(bilateral_filtered_image)

    # High Pass Filtering
    transformed_image1 = highPassFiltering(transformed_image)

    # Kalman Filtering
    transformed_image2 = kalman_filter(transformed_image1)

    # Gamma Transformation
    gamma_transformed_image = np.power(transformed_image2 / 255.0, gamma) * 255.0
    gamma_transformed_image = np.uint8(np.clip(gamma_transformed_image, 0, 255))

    # Wavelet Denoising
    denoised_image = wavelet_denoise(gamma_transformed_image)

    return denoised_image

# Loop over your dataset
for segments in traintest_folder:
    print("Working on {} ----------------------------------------------".format(segments))
    # Train Test Val Folders
    segment_path = join(input_path, segments)
    output_segment_path = join(output_path, segments)

    # Create Folder
    if not os.path.exists(output_segment_path):
        os.mkdir(output_segment_path)

    # Individual Folders
    classes = os.listdir(segment_path)
    for class_folder in classes:
        print("Working on {} ------------------".format(class_folder))
        input_class_folder = join(segment_path, class_folder)
        output_class_folder = join(output_segment_path, class_folder)

        # Create Folder
        if not os.path.exists(output_class_folder):
            os.mkdir(output_class_folder)

        images = os.listdir(input_class_folder)

        for i in images:
            # Read Image
            image = cv2.imread(join(input_class_folder, i), cv2.IMREAD_COLOR)

            # Preprocess the image with gamma transformation, wavelet denoising, and optimized bilateral filtering
            preprocessed_image = preprocess_image(image, gamma=1.5)

            filename = join(output_class_folder, i)
            cv2.imwrite("{}".format(filename), preprocessed_image)


Working on test ----------------------------------------------
Working on African Jambo ------------------
Working on Aprupali ------------------


C:\Users\Fahim\AppData\Local\Temp\ipykernel_17592\1767694625.py:87: RuntimeWarning: invalid value encountered in power
  gamma_transformed_image = np.power(transformed_image2 / 255.0, gamma) * 255.0
C:\Users\Fahim\AppData\Local\Temp\ipykernel_17592\1767694625.py:88: RuntimeWarning: invalid value encountered in cast
  gamma_transformed_image = np.uint8(np.clip(gamma_transformed_image, 0, 255))


Working on Baper Bari ------------------
Working on Bari 11 ------------------
Working on Bari 13 ------------------
Working on Bari 4 ------------------
Working on Bari 7 ------------------
Working on Bari 8 ------------------
Working on Bari 9 ------------------
Working on Fazlee ------------------
Working on Gopal Vog ------------------
Working on Haribhanga ------------------
Working on Him Sagor ------------------
Working on Indian Totapori ------------------
Working on Kacha Mitha ------------------
Working on King Breunei ------------------
Working on Lengra ------------------
Working on Maryam ------------------
Working on Modhurani ------------------
Working on Phillipine Honey Dew ------------------
